# Regresión lineal
En una regresión se tiene una variable objetivo $Y$ la cual es cuantitativa y es de interes para el investigador.
Se quiere construir una función $f(X)$ donde $X=(X_1, \ldots, X_p)$ es un conjunto de variables exogenas que se utilizaran para pronosticar a $Y$.

En un modelo de regresión lineal, se usan las funciones del tipo:
$$Y=\beta_0 +\beta_1X_1+\beta_2X_2+...+\beta_pX_p +\epsilon $$ 
o de la forma más general
$$f_0(Y)=\beta_0 +\beta_1 f_1(X_1)+\beta_2 f_2(X_2)+...+\beta_p f_p(X_p) +\epsilon $$ 
donde $\epsilon$ se conoce como el error o ruido del modelo.

Sobre este error se realizan varios supuestos para que el modelo tenga validez estadística.
1. Normalidad o gaussianidad
2. Homocedasticidad
3. Independencia


In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from scipy import stats
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm  ## Parte estadistica
from statsmodels.sandbox.regression.predstd import wls_prediction_std  ## Parte estadistica

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



## Datos
Los datos están [acá](https://archive.ics.uci.edu/ml/datasets/Communities+and+Crime+Unnormalized#) 


In [2]:
url="https://raw.githubusercontent.com/Cruzalirio/Ucentral/master/Bases/Violencia.csv"
violencia=pd.read_csv(url, sep=";", decimal=",", na_values="?",index_col=0)
violencia

,state,countyCode,communityCode,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,numbUrban,pctUrban,medIncome,pctWWage,pctWFarmSelf,pctWInvInc,pctWSocSec,pctWPubAsst,pctWRetire,medFamInc,perCapInc,whitePerCap,blackPerCap,indianPerCap,AsianPerCap,OtherPerCap,HispPerCap,NumUnderPov,PctPopUnderPov,PctLess9thGrade,PctNotHSGrad,PctBSorMore,PctUnemployed,PctEmploy,PctEmplManu,PctEmplProfServ,...,LemasTotalReq,LemasTotReqPerPop,PolicReqPerOffic,PolicPerPop,RacialMatchCommPol,PctPolicWhite,PctPolicBlack,PctPolicHisp,PctPolicAsian,PctPolicMinor,OfficAssgnDrugUnits,NumKindsDrugsSeiz,PolicAveOTWorked,LandArea,PopDens,PctUsePubTrans,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,LemasPctOfficDrugUn,PolicBudgPerPop,murders,murdPerPop,rapes,rapesPerPop,robberies,robbbPerPop,assaults,assaultPerPop,burglaries,burglPerPop,larcenies,larcPerPop,autoTheft,autoTheftPerPop,arsons,arsonsPerPop,ViolentCrimesPerPop,nonViolPerPop
communityname,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
BerkeleyHeightstownship,NJ,39.0,5320.0,1,11980,3.10,1.37,91.78,6.50,1.88,12.47,21.44,10.93,11.33,11980,100.00,75122,89.24,1.55,70.20,23.62,1.03,18.39,79584,29711,30233,13600,5725,27101,5115.0,22838,227,1.96,5.81,9.90,48.18,2.70,64.55,14.65,28.82,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.5,1845.9,9.63,NaN,NaN,NaN,NaN,0.0,NaN,0,0.00,0.0,0.00,1.0,8.20,4.0,32.81,14.0,114.85,138.0,1132.08,16.0,131.26,2.0,16.41,41.02,1394.59
Marpletownship,PA,45.0,47616.0,1,23123,2.82,0.80,95.57,3.44,0.85,11.01,21.30,10.48,17.18,23123,100.00,47917,78.99,1.11,64.11,35.50,2.75,22.85,55323,20148,20191,18137,0,20074,5250.0,12222,885,3.98,5.61,13.72,29.89,2.43,61.96,12.26,29.28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.6,2186.7,3.84,NaN,NaN,NaN,NaN,0.0,NaN,0,0.00,1.0,4.25,5.0,21.26,24.0,102.05,57.0,242.37,376.0,1598.78,26.0,110.55,1.0,4.25,127.56,1955.95
Tigardcity,OR,NaN,NaN,1,29344,2.43,0.74,94.33,3.43,2.35,11.36,25.88,11.01,10.28,29344,100.00,35669,82.00,1.15,55.73,22.25,2.94,14.56,42112,16946,17103,16644,21606,15528,5954.0,8405,1389,4.75,2.80,9.09,30.13,4.01,69.80,15.95,21.52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.6,2780.9,4.37,NaN,NaN,NaN,NaN,0.0,NaN,3,8.30,6.0,16.60,56.0,154.95,14.0,38.74,274.0,758.14,1797.0,4972.19,136.0,376.30,22.0,60.87,218.59,6167.51
Gloversvillecity,NY,35.0,29443.0,1,16656,2.40,1.70,97.35,0.50,0.70,12.55,25.20,12.19,17.57,0,0.00,20580,68.15,0.24,38.95,39.48,11.71,18.33,26501,10810,10909,9984,4941,3541,2451.0,4391,2831,17.23,11.05,33.68,10.81,9.86,54.74,31.22,27.43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.2,3217.7,3.31,NaN,NaN,NaN,NaN,0.0,NaN,0,0.00,10.0,57.86,10.0,57.86,33.0,190.93,225.0,1301.78,716.0,4142.56,47.0,271.93,NaN,NaN,306.64,NaN
Bemidjicity,MN,7.0,5068.0,1,11245,2.76,0.53,89.16,1.17,0.52,24.46,40.53,28.69,12.65,0,0.00,17390,69.33,0.55,42.82,32.16,11.21,14.43,24018,8483,9009,887,4425,3352,3000.0,1328,2855,29.99,12.15,23.06,25.28,9.08,52.44,6.89,36.54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.5,974.2,0.38,NaN,NaN,NaN,NaN,0.0,NaN,0,0.00,NaN,NaN,4.0,32.04,14.0,112.14,91.0,728.93,1060.0,8490.87,91.0,728.93,5.0,40.05,NaN,9988.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Mercedcity,CA,NaN,NaN,10,56216,3.07,6.87,61.68,15.23,29.86,15.46,30.16,14.34,8.08,56216,100.00,24727,75.05,1.12,31.42,21.45,19.98,14.41,27388,10237,13041,8344,8590,3399,6470.0,6644,13804,25.06,17.12,30.87,15.79,9.99,55.53,13.47,27.18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.7,3365.4,0.59,NaN,NaN,NaN,NaN,0.0,NaN,10,16.49,30.0,49.46,121.0,199.50,170.0,280.29,1376.0,2268.72,2563.0,4225.82

## Variable objetivo
En este caso tomaremos como $Y$ a la variable que indica el número de muertes por cada 100K habitantes.
Observamos las variables más relacionadas con la variable objetivo

In [3]:
## Selecciono las más correlacionadas con Y
a=violencia.corr()["murdPerPop"][np.abs(violencia.corr()["murdPerPop"])>0.5]
a

racepctblack           0.651688
racePctWhite          -0.666004
pctWPubAsst            0.520787
PctFam2Par            -0.613577
PctKids2Par           -0.640889
PctYoungKids2Par      -0.591170
PctTeen2Par           -0.590444
PctKidsBornNeverMar    0.679607
PctVacantBoarded       0.554762
PctPolicWhite         -0.561485
PctPolicBlack          0.752140
PctPolicMinor          0.596171
murdPerPop             1.000000
robbbPerPop            0.691058
assaultPerPop          0.559934
burglPerPop            0.549124
autoTheftPerPop        0.526832
ViolentCrimesPerPop    0.671541
Name: murdPerPop, dtype: float64

## Debemos tener cuidado con los faltantes

In [4]:
## En b quedan las variables que tienen menos de 10% de NA
b=violencia.isna().sum()[violencia.isna().sum()/violencia.shape[0]<0.1]
b/violencia.shape[0]

state                  0.000000
fold                   0.000000
population             0.000000
householdsize          0.000000
racepctblack           0.000000
                         ...   
autoTheftPerPop        0.001354
arsons                 0.041084
arsonsPerPop           0.041084
ViolentCrimesPerPop    0.099774
nonViolPerPop          0.043792
Length: 122, dtype: float64

## Selección de variables explicativas o exógenas

In [5]:
a.index

Index(['racepctblack', 'racePctWhite', 'pctWPubAsst', 'PctFam2Par',
       'PctKids2Par', 'PctYoungKids2Par', 'PctTeen2Par', 'PctKidsBornNeverMar',
       'PctVacantBoarded', 'PctPolicWhite', 'PctPolicBlack', 'PctPolicMinor',
       'murdPerPop', 'robbbPerPop', 'assaultPerPop', 'burglPerPop',
       'autoTheftPerPop', 'ViolentCrimesPerPop'],
      dtype='object')

In [6]:
## Me quedo con las variables que estan más correlacionadas con Y, y
## Tienen menos de 10% de datos faltantes
X=violencia[a.index[a.index.isin(b.index)]]
X=X.dropna() ### Quito datos faltantes
Y=X['murdPerPop']  ## Selecciono a Y
X=X.drop(["murdPerPop"], axis=1) ## Quito a Y de X

## División de los datos

En un conjunto de entrenamiento y uno de prueba, [documentación](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [7]:
X_train, X_test, y_train, y_test=train_test_split(X,Y, test_size=0.2)
print("Tamaño de X", X.shape)
print("Tamaño de X-train", X_train.shape)

Tamaño de X (1988, 14)
Tamaño de X-train (1590, 14)


In [8]:
reg=linear_model.LinearRegression()
reg.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

## Extracción de los parámetros

In [ ]:
pd.DataFrame(reg.coef_, index=X.columns)

,0
racepctblack,0.095035
racePctWhite,-0.085269
pctWPubAsst,0.137221
PctFam2Par,0.118738
PctKids2Par,-0.047970
PctYoungKids2Par,-0.014925
PctTeen2Par,-0.051285
PctKidsBornNeverMar,-0.192083
PctVacantBoarded,0.361180
robbbPerPop,-0.044408


## Metricas de evaluación

En este enlace están las metricas para la evaluación de modelos [link](https://scikit-learn.org/stable/modules/model_evaluation.html#explained-variance-score)

In [15]:
## Para una metrica (MAE) TRain data
from sklearn.metrics import mean_absolute_error, r2_score, median_absolute_error, mean_squared_error
ypron_train=reg.predict(X_train)
print("MAE=",mean_absolute_error(y_train, ypron_train))
print("R2=",r2_score(y_train, ypron_train))
print("P50Error=",median_absolute_error(y_train, ypron_train))
print("MSE=",mean_squared_error(y_train, ypron_train))

MAE= 3.7080595748721277
R2= 0.5944065612546179
P50Error= 2.237274880499516
MSE= 33.41444785168245


In [16]:
## Para una metrica (MAE) Test data
from sklearn.metrics import mean_absolute_error, r2_score, median_absolute_error, mean_squared_error
ypron_test=reg.predict(X_test)
print("MAE=",mean_absolute_error(y_test, ypron_test))
print("R2=",r2_score(y_test, ypron_test))
print("P50Error=",median_absolute_error(y_test, ypron_test))
print("MSE=",mean_squared_error(y_test, ypron_test))

MAE= 3.673261261116476
R2= 0.640567236030929
P50Error= 2.230708068995553
MSE= 31.583304406063224


In [ ]:
pred_train=reg.predict(X_train) ## Pronosticando dentro del entrenamiento
print("El R2 es ",100*np.round(r2_score(y_train, pred_train),4), "%")
fig=px.scatter(x=y_train, y=pred_train)
fig.show()

El R2 es  60.22 %


In [ ]:
pred_test=reg.predict(X_test)
print("El R2 es ",100*np.round(r2_score(y_test, pred_test),4), "%")
fig=px.scatter(x=y_test, y=pred_test)
fig.show()

El R2 es  61.59 %


## Problema de normalidad

Sí los datos son sesgados, la regresión puede tener problemas de pronostico, una solución es buscar una transformación para lograr suavizar los datos.
La transformación de Boxcox se realiza de manera potencial

$$Y_{N}=\frac{Y^\lambda-1}{\lambda}$$
Y su transformación inversa es:

$$Y=(\lambda Y_{N}+1)^{\frac{1}{\lambda}}$$

siempre y cuando $Y$ sea positiva, de lo contrario basta con restarle el mínimo de $Y$ más 1

In [ ]:
resid_train=y_train-pred_train ## Residuales de entrenamiento
fig=px.histogram(x=resid_train)
fig.show()

In [ ]:
y_train_bc,flambda = stats.boxcox(y_train+1)

# use lambda value to transform test data
y_test_bc = stats.boxcox(y_test+1, flambda)
reg=linear_model.LinearRegression()
reg.fit(X_train,y_train_bc)
pred_train_bc=reg.predict(X_train)
print("El R2 es ",100*np.round(r2_score(y_train_bc, pred_train_bc),4), "%")
pred_test_bc=reg.predict(X_test)
print("El R2 es ",100*np.round(r2_score(y_test_bc, pred_test_bc),4), "%")
resid_train_bc=y_train_bc-pred_train_bc
fig=px.histogram(x=resid_train_bc)
fig.show()

El R2 es  47.94 %
El R2 es  50.8 %


## Para análisis estadísticos


In [ ]:
model = sm.OLS(y_train, X_train) ### les debó un dulce
results = model.fit()
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:             murdPerPop   R-squared (uncentered):                   0.731
Model:                            OLS   Adj. R-squared (uncentered):              0.728
Method:                 Least Squares   F-statistic:                              305.4
Date:                Fri, 02 Oct 2020   Prob (F-statistic):                        0.00
Time:                        00:30:22   Log-Likelihood:                         -4995.5
No. Observations:                1590   AIC:                                  1.002e+04
Df Residuals:                    1576   BIC:                                  1.009e+04
Df Model:                          14                                                  
Covariance Type:            nonrobust                                                  
                          coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------

In [ ]:
SC=sum((pred_train)**2)
SCA=sum((y_train)**2)
SC/SCA

0.7574638200511533

## Selección de variables 

In [ ]:
a=violencia.corr()["murdPerPop"][np.abs(violencia.corr()["murdPerPop"])>0] ### Correlaciones
b=violencia.isna().sum()[violencia.isna().sum()/violencia.shape[0]<0.1] ## Nas
X=violencia[a.index[a.index.isin(b.index)]]
X=X.dropna()
Y=X['murdPerPop']
X=X.drop(["murdPerPop"], axis=1)
X

,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,numbUrban,pctUrban,medIncome,pctWWage,pctWFarmSelf,pctWInvInc,pctWSocSec,pctWPubAsst,pctWRetire,medFamInc,perCapInc,whitePerCap,blackPerCap,indianPerCap,AsianPerCap,OtherPerCap,HispPerCap,NumUnderPov,PctPopUnderPov,PctLess9thGrade,PctNotHSGrad,PctBSorMore,PctUnemployed,PctEmploy,PctEmplManu,PctEmplProfServ,PctOccupManu,PctOccupMgmtProf,MalePctDivorce,...,OwnOccLowQuart,OwnOccMedVal,OwnOccHiQuart,OwnOccQrange,RentLowQ,RentMedian,RentHighQ,RentQrange,MedRent,MedRentPctHousInc,MedOwnCostPctInc,MedOwnCostPctIncNoMtg,NumInShelters,NumStreet,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn,murders,rapes,rapesPerPop,robberies,robbbPerPop,assaults,assaultPerPop,burglaries,burglPerPop,larcenies,larcPerPop,autoTheft,autoTheftPerPop,arsons,arsonsPerPop,ViolentCrimesPerPop,nonViolPerPop
communityname,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
BerkeleyHeightstownship,1,11980,3.10,1.37,91.78,6.50,1.88,12.47,21.44,10.93,11.33,11980,100.00,75122,89.24,1.55,70.20,23.62,1.03,18.39,79584,29711,30233,13600,5725,27101,5115.0,22838,227,1.96,5.81,9.90,48.18,2.70,64.55,14.65,28.82,5.49,50.73,3.67,...,215900,262600,326900,111000,685,1001,1001,316,1001,23.8,21.1,14.0,11,0,10.66,53.72,65.29,78.09,89.14,6.5,1845.9,9.63,0.0,0,0.0,0.00,1.0,8.20,4.0,32.81,14.0,114.85,138.0,1132.08,16.0,131.26,2.0,16.41,41.02,1394.59
Marpletownship,1,23123,2.82,0.80,95.57,3.44,0.85,11.01,21.30,10.48,17.18,23123,100.00,47917,78.99,1.11,64.11,35.50,2.75,22.85,55323,20148,20191,18137,0,20074,5250.0,12222,885,3.98,5.61,13.72,29.89,2.43,61.96,12.26,29.28,6.39,37.64,4.23,...,136300,164200,199900,63600,467,560,672,205,627,27.6,20.7,12.5,0,0,8.30,77.17,71.27,90.22,96.12,10.6,2186.7,3.84,0.0,0,1.0,4.25,5.0,21.26,24.0,102.05,57.0,242.37,376.0,1598.78,26.0,110.55,1.0,4.25,127.56,1955.95
Tigardcity,1,29344,2.43,0.74,94.33,3.43,2.35,11.36,25.88,11.01,10.28,29344,100.00,35669,82.00,1.15,55.73,22.25,2.94,14.56,42112,16946,17103,16644,21606,15528,5954.0,8405,1389,4.75,2.80,9.09,30.13,4.01,69.80,15.95,21.52,8.79,32.48,10.10,...,74700,90400,112000,37300,370,428,520,150,484,24.1,21.7,11.6,16,0,5.00,44.77,36.60,61.26,82.85,10.6,2780.9,4.37,0.0,3,6.0,16.60,56.0,154.95,14.0,38.74,274.0,758.14,1797.0,4972.19,136.0,376.30,22.0,60.87,218.59,6167.51
Springfieldcity,1,140494,2.45,2.51,95.65,0.90,0.95,18.09,32.89,20.04,13.26,140494,100.00,21577,75.78,1.00,41.15,29.31,7.12,14.09,27705,11878,12029,7382,10264,10753,7192.0,8104,23223,17.78,8.76,23.03,20.66,5.72,59.02,14.31,26.83,14.72,23.42,11.40,...,37700,53900,73100,35400,215,280,349,134,340,26.4,17.3,11.7,327,4,1.49,64.35,42.29,70.61,85.66,70.4,1995.7,0.97,0.0,7,77.0,50.98,136.0,90.05,449.0,297.29,2094.0,1386.46,7690.0,5091.64,454.0,300.60,134.0,88.72,442.95,6867.42
Norwoodtown,1,28700,2.60,1.60,96.57,1.47,1.10,11.17,27.41,12.76,14.42,28700,100.00,42805,79.47,0.39,47.70,30.23,5.41,17.23,50394,18193,18276,17342,21482,12639,21852.0,22594,1126,4.01,4.49,13.89,27.01,4.85,65.42,14.02,27.17,8.50,32.78,5.97,...,155100,179000,215500,60400,463,669,824,361,736,24.4,20.8,12.5,0,0,9.19,77.30,63.45,82.23,93.53,10.9,2643.5,9.62,0.0,0,4.0,13.53,9.0,30.44,54.0,182.66,110.0,372.09,288.0,974.19,144.0,487.10,17.0,57.50,226.63,1890.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Mercedcity,10,56216,3.07,6.87,61.68,15.23,29.86,15.46,30.16,14.34,8.08,56216,100.00,24727,75.05,1.12,31.42,21.45,19.98,14.41,27388,10237,13041,8344,8590,3399,6470.0,6644,13804,25.06,17.12,30.87,15.79,9.99,55.53,13.47,27.18,16.38,25.02,10.22,...,71200,91100,118900,47700,298,374,45

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X,Y, test_size=0.2)
reg=linear_model.LinearRegression()
reg.fit(X_train,y_train)
pred_train=reg.predict(X_train)
print("El R2 de entrenamiento es ",100*np.round(r2_score(y_train, pred_train),4), "%")
pred_test=reg.predict(X_test)
print("El R2 de prueba es ",100*np.round(r2_score(y_test, pred_test),4), "%")

El R2 de entrenamiento es  100.0 %
El R2 de prueba es  100.0 %


In [ ]:
seleccion=RFE(reg,  n_features_to_select=4, step=1) ## Seleccionar un modelo más pequeño
seleccion = seleccion.fit(X_train, y_train)  ## Parsimonioso
X.columns[seleccion.support_]

Index(['rapesPerPop', 'robbbPerPop', 'assaultPerPop', 'ViolentCrimesPerPop'], dtype='object')

In [ ]:
pred_train=seleccion.predict(X_train)
print("El R2 de entrenamiento es ",100*np.round(r2_score(y_train, pred_train),4), "%")
pred_test=seleccion.predict(X_test)
print("El R2 de prueba es ",100*np.round(r2_score(y_test, pred_test),4), "%")
fig=px.scatter(X, x=y_train, y=X_train.ViolentCrimesPerPop)
fig.show()

El R2 de entrenamiento es  100.0 %
El R2 de prueba es  100.0 %


In [ ]:
pd.DataFrame(seleccion.ranking_, index=X.columns).sort_values(by=0).iloc[1:10] ## Ordenar las variables por importancia

,0
robbbPerPop,1
rapesPerPop,1
ViolentCrimesPerPop,1
MedNumBR,2
autoTheftPerPop,3
nonViolPerPop,4
larcPerPop,5
burglPerPop,6
arsonsPerPop,7


## Variables Dummy
Una variable dummy es una variable que surge de una variable categorica


In [ ]:
url="https://raw.githubusercontent.com/Cruzalirio/Ucentral/master/Bases/Violencia.csv"
violencia=pd.read_csv(url, sep=";", decimal=",", na_values="?",index_col=0)
violencia=pd.get_dummies(violencia, drop_first=True) ## variables dummy
a=violencia.corr()["murdPerPop"][np.abs(violencia.corr()["murdPerPop"])>0] ## Correlacion
b=violencia.isna().sum()[violencia.isna().sum()/violencia.shape[0]<0.1] ### las de NA
X=violencia[a.index[a.index.isin(b.index)]]
X=X.dropna() ## Quite los NAs
Y=X['murdPerPop']
X=X.drop(["murdPerPop"], axis=1)
X

,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,numbUrban,pctUrban,medIncome,pctWWage,pctWFarmSelf,pctWInvInc,pctWSocSec,pctWPubAsst,pctWRetire,medFamInc,perCapInc,whitePerCap,blackPerCap,indianPerCap,AsianPerCap,OtherPerCap,HispPerCap,NumUnderPov,PctPopUnderPov,PctLess9thGrade,PctNotHSGrad,PctBSorMore,PctUnemployed,PctEmploy,PctEmplManu,PctEmplProfServ,PctOccupManu,PctOccupMgmtProf,MalePctDivorce,...,state_DE,state_FL,state_GA,state_IA,state_ID,state_IL,state_IN,state_KS,state_KY,state_LA,state_MA,state_MD,state_ME,state_MI,state_MN,state_MO,state_MS,state_NC,state_ND,state_NH,state_NJ,state_NM,state_NV,state_NY,state_OH,state_OK,state_OR,state_PA,state_RI,state_SC,state_SD,state_TN,state_TX,state_UT,state_VA,state_VT,state_WA,state_WI,state_WV,state_WY
communityname,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
BerkeleyHeightstownship,1,11980,3.10,1.37,91.78,6.50,1.88,12.47,21.44,10.93,11.33,11980,100.00,75122,89.24,1.55,70.20,23.62,1.03,18.39,79584,29711,30233,13600,5725,27101,5115.0,22838,227,1.96,5.81,9.90,48.18,2.70,64.55,14.65,28.82,5.49,50.73,3.67,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Marpletownship,1,23123,2.82,0.80,95.57,3.44,0.85,11.01,21.30,10.48,17.18,23123,100.00,47917,78.99,1.11,64.11,35.50,2.75,22.85,55323,20148,20191,18137,0,20074,5250.0,12222,885,3.98,5.61,13.72,29.89,2.43,61.96,12.26,29.28,6.39,37.64,4.23,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
Tigardcity,1,29344,2.43,0.74,94.33,3.43,2.35,11.36,25.88,11.01,10.28,29344,100.00,35669,82.00,1.15,55.73,22.25,2.94,14.56,42112,16946,17103,16644,21606,15528,5954.0,8405,1389,4.75,2.80,9.09,30.13,4.01,69.80,15.95,21.52,8.79,32.48,10.10,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
Springfieldcity,1,140494,2.45,2.51,95.65,0.90,0.95,18.09,32.89,20.04,13.26,140494,100.00,21577,75.78,1.00,41.15,29.31,7.12,14.09,27705,11878,12029,7382,10264,10753,7192.0,8104,23223,17.78,8.76,23.03,20.66,5.72,59.02,14.31,26.83,14.72,23.42,11.40,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Norwoodtown,1,28700,2.60,1.60,96.57,1.47,1.10,11.17,27.41,12.76,14.42,28700,100.00,42805,79.47,0.39,47.70,30.23,5.41,17.23,50394,18193,18276,17342,21482,12639,21852.0,22594,1126,4.01,4.49,13.89,27.01,4.85,65.42,14.02,27.17,8.50,32.78,5.97,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Mercedcity,10,56216,3.07,6.87,61.68,15.23,29.86,15.46,30.16,14.34,8.08,56216,100.00,24727,75.05,1.12,31.42,21.45,19.98,14.41,27388,10237,13041,8344,8590,3399,6470.0,6644,13804,25.06,17.12,30.87,15.79,9.99,55.53,13.47,27.18,16.38,25.02,10.22,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Pinevillecity,10,12251,2.68,21.18,76.65,1.52,1.29,17.36,31.23,16.97,12.57,12251,100.00,20321,75.06,0.47,33.25,27.63,8.85,18.23,25000,9995,11353,5768,10910,1718,11471.0,4883,2364,20.79,12.51,27.71,19.28,7.90,54.64,7.81,37.06,10.37,28.73,10.86,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Yucaipacity,10,32824,2.46,0.52,92.62,0.98,11.00,11.81,20.96,9.53,20.73,32824,100.00,27182,59.79,0.51,44.72,43.40,9.01,23.56,34973,14131,14416,13630,13197,17313,8532.0,9398,2460,7.56,7.82,26.14,12.42,5.18,50.54,9.34,23.36,13.53,23.54,9.89,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X,Y, test_size=0.2)
reg=linear_model.LinearRegression()
reg.fit(X_train,y_train)
seleccion=RFE(reg,  n_features_to_select=4, step=1)
seleccion = seleccion.fit(X_train, y_train)
pd.DataFrame(seleccion.ranking_, index=X.columns).sort_values(by=0).iloc[1:40]

,0
assaultPerPop,1
ViolentCrimesPerPop,1
robbbPerPop,1
arsonsPerPop,2
burglPerPop,3
larcPerPop,4
autoTheftPerPop,5
nonViolPerPop,6
state_DC,7
state_SD,8


In [ ]:
pred_train=seleccion.predict(X_train)
print("El R2 de entrenamiento es ",100*np.round(r2_score(y_train, pred_train),4), "%")
pred_test=seleccion.predict(X_test)
print("El R2 de prueba es ",100*np.round(r2_score(y_test, pred_test),4), "%")

El R2 de entrenamiento es  100.0 %
El R2 de prueba es  100.0 %
